Ordering and Delivery to Google Earth Engine (GEE)

Based off of the client's example (https://github.com/planetlabs/planet-client-python/blob/main/examples/orders_create_and_download_multiple_orders.py)

In [ ]:
import planet

Define order

In [ ]:
iowa_aoi = {
    "type":
    "Polygon",
    "coordinates": [[[-91.198465, 42.893071], [-91.121931, 42.893071],
                     [-91.121931, 42.946205], [-91.198465, 42.946205],
                     [-91.198465, 42.893071]]]
}

iowa_images = ['20200925_161029_69_2223', '20200925_161027_48_2223']

Define cloud delivery location

In [ ]:
delivery_config = planet.order_request.delivery(
    cloud_config=planet.order_request.google_earth_engine(
        project='planet-devrel-dev', collection='gee-integration-testing'))
print(delivery_config)

Build order request

In [ ]:
data_products = [
    planet.order_request.product(iowa_images, 'analytic_udm2', 'PSScene')
]
tools = [planet.order_request.clip_tool(iowa_aoi)]

iowa_order = planet.order_request.build_request(name='iowa_order',
                                                products=data_products,
                                                delivery=delivery_config,
                                                tools=tools)

print(iowa_order)

Define create and download order functions

In [ ]:
async def create_and_get_order(order_detail, client):
    with planet.reporting.StateBar(state='creating') as reporter:
        # create
        order = await client.create_order(order_detail)
        reporter.update(state='created', order_id=order['id'])
        # wait for completion
        await client.wait(order['id'], callback=reporter.update_state)

    order_details = await client.get_order(order_id=order['id'])
    return order_details

Create and download order

In [ ]:
async with planet.Session() as ps:
    client = planet.OrdersClient(ps)
    order_details = await create_and_get_order(iowa_order, client)

In [ ]:
print(order_details)